In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import os
import yaml
import numpy as np

In [ ]:
from distgen import Generator

In [ ]:
input_yaml = """
n_particle: 100000
species: electron
start: 
  type: free
random:
  type: hammersley
total_charge:
  units: pC
  value: 10
r_dist:
  max_r:
    units: mm
    value: 1
  type: radial_uniform
p_dist:
  sigma_p:
    value: 1
    units: eV/c
  avg_p: 
    value: 100
    units: eV/c
  type: gaussian
"""

In [ ]:
gen = Generator(input_yaml, verbose=True)

In [ ]:
P = gen.run()

In [ ]:
P.plot("p")

In [ ]:
P.plot("x", "px")

In [ ]:
P.plot("pz")

In [ ]:
P.plot("px")

In [ ]:
P.plot("py")

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

px, py, pz = P["px"], P["py"], P["pz"]

p = np.sqrt(px**2 + py**2 + pz**2)

theta = np.arctan2(py, px)
phi = np.arccos(pz / p)
hist, pedges = np.histogram(phi, bins=100, density=True)
pcs = (pedges[1:] + pedges[:-1]) / 2

x = px / p
y = py / p
z = pz / p

ax.scatter(x[::200], y[::200], z[::200], ".")
ax.set_xlabel(r"$\hat{p}_x$")
ax.set_ylabel(r"$\hat{p}_y$")
ax.set_zlabel(r"$\hat{p}_z$")

In [ ]:
theta = np.arctan2(py, px)
phi = np.arccos(pz / p)
hist, pedges = np.histogram(phi, bins=100, density=True)
pcs = (pedges[1:] + pedges[:-1]) / 2

plt.plot(pcs, hist);

In [ ]:
rns = np.random.random(100000)

In [ ]:
pa = 0
pb = np.pi

Ca = np.cos(pa)
Cb = np.cos(pb)

In [ ]:
rho = 1 / (Ca - Cb)

In [ ]:
phis = np.linspace(pa, pb, 1000)

In [ ]:
plt.plot(phis, rho * np.sin(phis));

In [ ]:
np.trapezoid(rho * np.sin(phis), phis)

In [ ]:
cdf = (Ca - np.cos(phis)) * rho

In [ ]:
from scipy.integrate import cumulative_trapezoid as cumtrapz

In [ ]:
plt.plot(phis, cdf, phis, cumtrapz(rho * np.sin(phis), phis, initial=0));

In [ ]:
ps = np.arccos(Ca - rns * (Ca - Cb))

In [ ]:
hist, pedges = np.histogram(ps, bins=30, density=True)

In [ ]:
pcs = (pedges[1:] + pedges[:-1]) / 2

In [ ]:
plt.plot(pcs, hist, phis, rho * np.sin(phis));

In [ ]:
gen = Generator("data/beer.can.in.yaml", verbose=1)

# print(gen)

In [ ]:
P1 = gen.run()

In [ ]:
P1.plot("x", "px")

In [ ]:
P1.plot("p")

In [ ]:
P1.plot("pz")

In [ ]:
P1["sigma_px"], P1["sigma_py"], P1["sigma_pz"]

In [ ]:
gen = Generator("data/maxwell_boltzmann.beer.can.in.yaml", verbose=1)

In [ ]:
P2 = gen.run()

In [ ]:
P2["sigma_px"], P2["sigma_py"], P2["sigma_pz"]

In [ ]:
gen = Generator("data/maxwell_boltzmann_KE.beer.can.in.yaml", verbose=1)

In [ ]:
P3 = gen.run()

In [ ]:
P3.plot("kinetic_energy")

In [ ]:
P3.plot("x", "px")

In [ ]:
P3["sigma_px"], P3["sigma_py"], P3["sigma_pz"]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

px, py, pz = P3["px"], P3["py"], P3["pz"]

p = np.sqrt(px**2 + py**2 + pz**2)

theta = np.arctan2(py, px)
phi = np.arccos(pz / p)
hist, pedges = np.histogram(phi, bins=100, density=True)
pcs = (pedges[1:] + pedges[:-1]) / 2

x = px / p
y = py / p
z = pz / p

ax.scatter(x[::500], y[::500], z[::500], ".")
ax.set_xlabel(r"$\hat{p}_x$")
ax.set_ylabel(r"$\hat{p}_y$")
ax.set_zlabel(r"$\hat{p}_z$")

In [ ]:
KE = np.linspace(0, 150, 10000)

E0 = 0

A1 = (0.8,)
m1 = 8

A2 = 0.1
m2 = 90

PKE = A1 * np.exp(-np.abs(KE - E0) / m1) + A2 * np.exp(-np.abs(KE - E0) / m2)
PKE = PKE / np.trapz(PKE, KE)  # Numerically intergate to normalize

plt.plot(KE, PKE)
plt.xlabel("KE (eV)")
plt.ylabel("$\\rho(KE)$ (1/eV)");

In [ ]:
dat = np.zeros((len(KE), 2))
dat[:, 0], dat[:, 1] = KE, PKE

np.savetxt("KEdist.txt", dat, header="KE     PKE", comments="")

In [ ]:
q = 1000 * 1.60217663e-19

print(q)

input_yaml = """
n_particle: 1000
species: electron
start: 
  type: cathode
random:
  type: hammersley
total_charge:
  units: C
  value: 1.60217663e-17
r_dist:
  max_r:
    units: mm
    value: 14.6
  type: radial_uniform
KE_dist:
  file: KEdist.txt
  units: eV
  type: file1d
transforms:
  sx:
    avg_x:
      units: millimeter
      value: -50
    type: set_avg x
  sy:
    avg_y:
      units: millimeter
      value: 50
    type: set_avg y
  sz:
    avg_z:
      units: millimeter
      value: 65
    type: set_avg z
output:
    file: None
"""

In [ ]:
inputs = yaml.safe_load(input_yaml)

n = 1000

inputs["n_particle"] = n
inputs["output"]["file"] = "test_ion_writer.ion"
inputs["output"]["type"] = "simion"
inputs["total_charge"]["value"] = n * 1.60217663e-19
inputs["transforms"]["sx"]["avg_x"]["value"] = -50
inputs["transforms"]["sy"]["avg_y"]["value"] = 50
inputs["transforms"]["sz"]["avg_z"]["value"] = 55

In [ ]:
gen = Generator(inputs, verbose=1)

In [ ]:
P = gen.run()

In [ ]:
P.plot("x", "y", figsize=(5, 5))

In [ ]:
P.plot("x", "px", figsize=(5, 5))

In [ ]:
P.plot("kinetic_energy")

In [ ]:
from distgen.writers import writer

writer("simion", gen.beam(), "test_ion_writer.ion", params={"color": 0})

In [ ]:
from scipy.constants import physical_constants

mc2 = 1e6 * physical_constants["electron mass energy equivalent in MeV"][0]
e_ = physical_constants["elementary charge"][0]
me = physical_constants["electron mass in u"][0]


def particle_group_to_SIMION(P, filename, color=0):
    header = ";0"

    simion_params = [
        "TOB",
        "MASS",
        "CHARGE",
        "X",
        "Y",
        "Z",
        "AZ",
        "EL",
        "KE",
        "CWF",
        "COLOR",
    ]

    # simion_units = {
    #    "TOB": "usec",
    #    "MASS": "amu",
    #    "CHARGE": "e",
    #    "X": "mm",
    #    "Y": "mm",
    #    "Z": "mm",
    #    "AZ": "deg",
    #    "EL": "deg",
    #    "CWF": "",
    #    "COLOR": "",
    # }

    data = np.zeros((len(P), len(simion_params)))

    data[:, simion_params.index("TOB")] = P.t * 1e6  # [P.t] = sec, convert to usec

    if P.species == "electron":
        data[:, simion_params.index("MASS")] = np.full(len(P), me)
        data[:, simion_params.index("CHARGE")] = np.full(len(P), -1)
    else:
        raise ValueError(f"Species {P.species} is not supported")

    data[:, simion_params.index("X")] = P.z * 1e3
    data[:, simion_params.index("Y")] = P.y * 1e3
    data[:, simion_params.index("Z")] = -P.x * 1e3

    px = P.pz
    py = P.py
    pz = -P.px

    data[:, simion_params.index("KE")] = P.kinetic_energy  # [eV]
    data[:, simion_params.index("AZ")] = np.arctan2(-pz, px) * (180 / np.pi)  # [deg]
    data[:, simion_params.index("EL")] = np.arctan2(py, np.sqrt(px**2 + pz**2)) * (
        180 / np.pi
    )  # [deg]

    data[:, simion_params.index("CWF")] = (
        P.weight / e_
    )  # Charge Weighting Factor, derive from particle group weights
    data[:, simion_params.index("COLOR")] = np.full(len(P), color)
    # fname, X, fmt='%.18e', delimiter=' '

    np.savetxt(filename, data, delimiter=",", header=header, comments="", fmt="  %.9e")

In [ ]:
particle_group_to_SIMION(P, "text_ion_file.ion")

In [ ]:
def read_simion_ION_file(filename):
    data = np.loadtxt(filename, comments=";", delimiter=",", skiprows=1)

    simion_params = [
        "TOB",
        "MASS",
        "CHARGE",
        "X",
        "Y",
        "Z",
        "AZ",
        "EL",
        "KE",
        "CWF",
        "COLOR",
    ]

    return {simion_params[ii]: data[:, ii] for ii, p in enumerate(simion_params)}

In [ ]:
ions1 = read_simion_ION_file("text_ion_file.ion")
ions2 = read_simion_ION_file("test_ion_writer.ion")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

px, py, pz = P["px"], P["py"], P["pz"]

p = np.sqrt(px**2 + py**2 + pz**2)

theta = np.arctan2(py, px)
phi = np.arccos(pz / p)
hist, pedges = np.histogram(phi, bins=100, density=True)
pcs = (pedges[1:] + pedges[:-1]) / 2

x = px / p
y = py / p
z = pz / p

ax.scatter(x[::], y[::], z[::], ".")
ax.set_xlabel(r"$\hat{p}_x$")
ax.set_ylabel(r"$\hat{p}_y$")
ax.set_zlabel(r"$\hat{p}_z$")

In [ ]:
for p in ions1:
    print(f"{p}:", max(np.abs(ions1[p] - ions2[p])))

In [ ]:
plt.plot(ions1["KE"], ions1["KE"] / ions2["KE"], ".")
plt.xlabel("KE (eV)");

In [ ]:
os.remove("text_ion_file.ion")
os.remove("KEdist.txt")
os.remove("test_ion_writer.ion")